In [328]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")

In [329]:
import pandas as pd
import pmdarima as pm

from matplotlib import pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')

---
### Data processing

In [333]:
df = pd.read_csv('data/pizza-sales-data.csv', parse_dates=['DATE'])

In [334]:
df.sort_values(['CATEGORY', 'DATE'], inplace=True)

In [335]:
df.reset_index(drop=True, inplace=True)

---
### Data modelling using `pmdarima`

In [336]:
def predict_data(train: pd.Series, n_periods: int):
    try:
        arima = pm.auto_arima(train, seasonal=False, error_action='ignore')
        prediction = arima.predict(n_periods=n_periods)
    except Exception as ex:
        print(str(ex))
        prediction = None
    finally:
        return prediction

In [337]:
step = int(len(data) * .85) # %85 for train

In [339]:
first_time = True
for idx, data in df.groupby(['CATEGORY']):
    data.set_index(['DATE'], inplace=True)
    
    train, test = data['SALES'][:step], data['SALES'][step:]
    predicted = predict_data(train=train, n_periods=test.shape[0])
    data['SALES_PRED'] = pd.Series(data=predicted, index=test.index, name=test.name)
    
    if first_time:
        data.to_csv('export/pizza-sales-data-prediction-result.csv')
        first_time = False
    else:
        data.to_csv('export/pizza-sales-data-prediction-result.csv', mode='a', header=False)
    